In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define symbolic variables
var('a b c')
assume(a > 0, b > 0, c > 0)

def get_reps(k):
    """
    Returns the su(2) representation matrices E1, E2, E3 for weight k.
    Dimension of the matrix is k+1.
    """
    dim = k + 1
    # Initialize matrices in Symbolic Ring (SR)
    E1 = Matrix(SR, dim, dim, 0)
    E2 = Matrix(SR, dim, dim, 0)
    E3 = Matrix(SR, dim, dim, 0)
    
    for l in range(dim):
        # E1: Diagonal elements (weights)
        E1[l, l] = I * (k - 2*l)
        
        # E2: Real part of ladder operators
        if l > 0:
            E2[l-1, l] = -l           # Contribution from P_{l-1}
        if l < k:
            E2[l+1, l] = k - l        # Contribution from P_{l+1}
        
        # E3: Imaginary part of ladder operators
        if l > 0:
            E3[l-1, l] = -l * I
        if l < k:
            E3[l+1, l] = -(k - l) * I
            
    return E1, E2, E3

def get_curv(a, b, c):
    """
    Calculates the eigenvalues r_ij of the curvature operator R on 2-vectors.
    Note: Based on the definition R(X^Y) = 0.5 * sum(E_i ^ R(X,Y)E_i),
    these eigenvalues are NEGATIVE for the standard sphere (a=b=c=1).
    """
    r12 = 3*a**2*b**2/c**2 - a**2*c**2/b**2 - b**2*c**2/a**2 - 2*a**2 - 2*b**2 + 2*c**2
    r13 = -a**2*b**2/c**2 + 3*a**2*c**2/b**2 - b**2*c**2/a**2 - 2*a**2 + 2*b**2 - 2*c**2
    r23 = -a**2*b**2/c**2 - a**2*c**2/b**2 + 3*b**2*c**2/a**2 + 2*a**2 - 2*b**2 - 2*c**2
    return r12, r13, r23

def get_Delta(k, a_val=None, b_val=None, c_val=None, numeric=False):
    """
    Constructs the Hodge-Laplacian matrix Delta for representation k.
    If a_val, b_val, c_val are provided, substitutes them.
    If numeric=True, converts to RDF (Real Double Field) for speed.
    """
    # 1. Get representation matrices
    E1, E2, E3 = get_reps(k)
    dim_rep = E1.nrows()
    
    # Identity matrices
    id_rep = identity_matrix(SR, dim_rep)
    id_form = identity_matrix(SR, 3) # 1-forms are 3D
    
    # 2. Connection matrices N (Levi-Civita on 1-forms)
    # Note: N_i represents -Gamma_{i, .}^.
    # Terms like (ab/c + ac/b - bc/a) appear frequently.
    
    N1 = -1 * Matrix(SR, [
        [0, 0, 0],
        [0, 0, -(-a*b/c - a*c/b + b*c/a)],
        [0, -a*b/c - a*c/b + b*c/a, 0]
    ])
    
    N2 = -1 * Matrix(SR, [
        [0, 0, -(a*b/c - a*c/b + b*c/a)],
        [0, 0, 0],
        [a*b/c - a*c/b + b*c/a, 0, 0]
    ])
    
    N3 = -1 * Matrix(SR, [
        [0, -(a*b/c - a*c/b - b*c/a), 0],
        [a*b/c - a*c/b - b*c/a, 0, 0],
        [0, 0, 0]
    ])
    
    # 3. Casimir Operators
    # Casimir on V_k (E_i are anti-hermitian, so squares are negative)
    Cas_rep = (a**2 * E1*E1 + b**2 * E2*E2 + c**2 * E3*E3)
    
    # Casimir on 1-forms
    Cas_form = (N1*N1 + N2*N2 + N3*N3).apply_map(lambda x: x.expand())
    
    # Mixed interaction term
    Mixed = 2*a*E1.tensor_product(N1) + 2*b*E2.tensor_product(N2) + 2*c*E3.tensor_product(N3)
    
    # 4. Curvature term q(R) (Ricci)
    r12, r13, r23 = get_curv(a, b, c)
    
    # q(R) acts diagonally. Since r_ij are negative, (-r_ij - r_ik) yields positive Ricci curvature.
    qR = diagonal_matrix([-r12-r13, -r23-r12, -r23-r13])
    
    # 5. Assemble Delta = nabla*nabla + 2q(R)
    # Rough Laplacian = -(Cas_rep + Cas_form + Mixed)
    RoughLaplacian = -(Cas_rep.tensor_product(id_form) + id_rep.tensor_product(Cas_form) + Mixed)
    CurvatureTerm = id_rep.tensor_product(qR)
    
    Delta = RoughLaplacian + CurvatureTerm
    
    # 6. Substitution / Numeric conversion
    if a_val is not None and b_val is not None and c_val is not None:
        Delta = Delta.subs(a=a_val, b=b_val, c=c_val)
        if numeric:
            Delta = Delta.change_ring(RDF) 
            
    return Delta